# Case-study: Impact of SAF attribute selection

In [1]:
# Get python executable for jupyter notebook
import sys
print(sys.executable)

/opt/homebrew/opt/python@3.11/bin/python3.11


In [64]:
!safinfer -h

usage: safinfer [-h] [-i DIR_IN] [-a ARCH] [-m MAP] [-p PROB] [-s SPARSEOPTS]
                [-o DIR_OUT] [-b BINDING_OUT] [-t TAXO_SCRIPT_LIB]
                [-T TOPOLOGY_OUT] [-r] [-L] [-f LOG_FILE] [-q SETTINGS]

options:
  -h, --help            show this help message and exit
  -i DIR_IN, --dir-in DIR_IN
  -a ARCH, --arch ARCH
  -m MAP, --map MAP
  -p PROB, --prob PROB
  -s SPARSEOPTS, --sparseopts SPARSEOPTS
  -o DIR_OUT, --dir-out DIR_OUT
  -b BINDING_OUT, --binding-out BINDING_OUT
  -t TAXO_SCRIPT_LIB, --taxo-script-lib TAXO_SCRIPT_LIB
  -T TOPOLOGY_OUT, --topology-out TOPOLOGY_OUT
  -r, --reconfigurable-arch
  -L, --log
  -f LOG_FILE, --log-file LOG_FILE
  -q SETTINGS, --settings SETTINGS
                        safinfer configuration file.


In [ ]:
!rm -f ref_output/new_arch.yaml ref_output/new_arch.yaml.* ref_output/bindings.yaml 
!safinfer -a ref_input/arch.yaml -s ref_input/sparseopts_C_lf.yaml -q ref_input/safinfer_settings_C_lf.yaml -o ref_output/ -L
!mv ref_output/new_arch.yaml.pretty ref_output/new_arch_C_lf.yaml.pretty
!mv ref_output/new_arch.yaml ref_output/new_arch_C_lf.yaml
!mv ref_output/bindings.yaml ref_output/bindings_C_bidir.yaml
!mv safinfer.log safinfer_C_lf.log

In [ ]:
!safmodel -h

In [66]:
!rm -f ref_output/new_arch.yaml ref_output/new_arch.yaml.*
!safinfer -a ref_input/arch.yaml -s ref_input/sparseopts_C_bidir.yaml -q ref_input/safinfer_settings_C_bidir.yaml -o ref_output/ -L
!mv ref_output/new_arch.yaml.pretty ref_output/new_arch_C_bidir.yaml.pretty
!mv ref_output/new_arch.yaml ref_output/new_arch_C_bidir.yaml
!mv safinfer.log safinfer_C_bidir.log

zsh:1: no matches found: ref_output/new_arch.yaml.*


- SAFinfer settings path: ref_input/safinfer_settings_C_bidir.yaml
logging: True
>> SAFinfer
:: Setup
:: => Done, setup
reconfigurable_arch: False
:: Taxonomic inference
- COMPLETE: microarchitecture inference
  => SUCCESS
- Dumping inferred SAF microarchitecture topology to ref_output/new_arch.yaml ...
:: => Done, taxonomic inference
<< Done, SAFinfer


# Fake SAFModel

In [92]:
import safmodel
import core.helper as helper
from core.helper import info,warn,error
from core import safmodel_core as safcore, \
                 safmodel_io as safio, \
                 safinfer_io
import os
import dill, yaml
import copy
import sympy as sp
import core.model.CasCompat as cc_
import math
from tqdm import tqdm
import numpy as np
import pandas as pd

In [2]:
# Spoof CLI flag parsing

import sys

sys.argv = [
    'safmodel', # This is typically the script name
    '-T', 'ref_output/new_arch_C_bidir.yaml',
    '-a', 'ref_input/arch.yaml',
    '-s', 'ref_input/sparseopts_C_bidir.yaml',
    '-c', 'ref_input/compound_components.yaml',
    '-U', 'ref_input/safmodel_settings.yaml',
    '-r', 'ref_output/arch_w_SAF.yaml',
    '-k', 'ref_output/',
    '-L'
]

arch, \
taxo_uarch, \
sparseopts, \
comp_in, \
arch_out_path, \
comp_out_path, \
user_attributes, \
do_logging,\
log_fn, \
characterization_path_list, \
model_script_lib_list, \
taxo_script_lib_list = safio.parse_args()

safmodel.log_config(do_logging,log_fn)
safmodel.opening_remark()
safmodel.setup(taxo_script_lib_list,characterization_path_list,model_script_lib_list)

SAFmodel.

Parsing input files:
- netlist: ref_output/new_arch_C_bidir.yaml
- arch: ref_input/arch.yaml
- sparseopts: ref_input/sparseopts_C_bidir.yaml
- compound components (input): ['ref_input/compound_components.yaml']
- arch output path: ref_output/arch_w_SAF.yaml
- compound components path (output): ref_output/
- SAFModel settings path: ref_input/safmodel_settings.yaml
logging: True
>> SAFmodel
:: Setup


In [3]:
# Build scale inference problem

warn(":: Scale inference",also_stdout=True)
'''Build scale inference problem'''
scale_prob=safcore.build_scale_inference_problem(arch, sparseopts, taxo_uarch, user_attributes=user_attributes)

:: Scale inference
Building scale inference problem...
Building RegisteredEqualityComparator_bitwidth$(u) energy area latency table.: 100%|██████████| 7/7 [00:00<00:00, 162.32it/s]
- RegisteredEqualityComparator_bitwidth$(u) MSE energy error (symbolic vs numerical): 1.525611648803889e-31
- RegisteredEqualityComparator_bitwidth$(u) RNMSE energy error (numerical vs true): 8.4 %
- RegisteredEqualityComparator_bitwidth$(u) MSE area error (symbolic vs numerical): 9.707226444223633e-25
- RegisteredEqualityComparator_bitwidth$(u) RNMSE area error (numerical vs true): 2.6 %
Building RegisteredEqualityComparator_bitwidth$(u) energy area latency table.: 100%|██████████| 7/7 [00:00<00:00, 191.93it/s]
- RegisteredEqualityComparator_bitwidth$(u) MSE energy error (symbolic vs numerical): 1.525611648803889e-31
- RegisteredEqualityComparator_bitwidth$(u) RNMSE energy error (numerical vs true): 8.4 %
- RegisteredEqualityComparator_bitwidth$(u) MSE area error (symbolic vs numerical): 9.707226444223633e-

In [4]:
list(scale_prob.keys())

['symbols',
 'symbol_types',
 'constraints',
 'energy_objectives',
 'area_objectives',
 'yields',
 'primitive_models',
 'component_models',
 'buffer_action_tree',
 'component_energy_action_tree',
 'sub_action_graph',
 'global_objective',
 'global_energy_objective',
 'global_area_objective',
 'abstract_global_objective_expression',
 'area_multiplier_dict',
 'simplified_symbols',
 'simplified_symbol_types',
 'simplified_constraints',
 'user_attributes']

In [5]:
scale_prob['global_objective']

'(0.0238329184902302*TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_rw_thresh**3 - 0.005111680555555566*TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_rw_thresh**2*TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_ww_thresh - 0.03860355925925933*TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_rw_thresh**2*TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_out_rw_thresh - 0.111935355357537*TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_rw_thresh**2 + 0.00045430396341473*TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_rw_thresh*TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_ww_thresh**2 + 0.0113931816666667*TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_rw_thresh*TestArchitecture.Skipping_shared_buffer3_shared_buffer1.Intersec

In [6]:
scale_prob_filename = 'scale_prob.dill'
simplified_constraints_filename = 'simplified_constraints.txt'
simplified_symbols_filename = 'simplified_symbols.txt'
simplified_symbol_types_filename = 'simplified_symbol_types.txt'
global_objective_filename = 'global_objective_file.yaml'
#area_objectives_filename = 'area_objectives_file.yaml'

# Dumping scale problem structure

import dill
# Open the file in binary write mode and save the variable
with open(scale_prob_filename, 'wb') as file:
    dill.dump(scale_prob, file)

print(f"Variable 'scale_prob' has been saved to {scale_prob_filename}")

# Dumping 

# Assuming scale_prob is your dictionary and 'simplified_constraints' is the key
simplified_constraints = scale_prob['simplified_constraints']
simplified_symbols = scale_prob['simplified_symbols']
simplified_symbol_types = scale_prob['simplified_symbol_types']
global_objective = scale_prob['global_objective']
#area_objectives = scale_prob['area_objectives']

# Specify the filename for the text file


# Constraints
with open(simplified_constraints_filename, 'w') as file:
    for constraint in simplified_constraints:
        file.write(constraint + '\n')

# Symbols
with open(simplified_symbols_filename, 'w') as file:
    for symbol in simplified_symbols:
        file.write(symbol + '\n')

# Symbol types
with open(simplified_symbol_types_filename, 'w') as file:
    for symbol_type in simplified_symbol_types:
        file.write(symbol_type + '\n')

import yaml
# Energy objectives dump
with open(global_objective_filename, 'w') as file:
    yaml.dump(global_objective, file)


#print(f"List 'simplified_constraints' has been saved to {simplified_symbol_types_filename}")


Variable 'scale_prob' has been saved to scale_prob.dill


## Routines which support modifying the constraint system

In [7]:
def load_lines_from_txt_file(filename):
    with open(filename, 'r') as file:
        return [line.strip() for line in file]
    
def load_objective_from_yaml_file(filename):
    with open(filename, 'r') as file:
        return yaml.safe_load(file)

In [8]:
def load_C_C_bidir_skip_constraint_system(modified_constraints_filename = "modified_constraints.txt"):
    scale_prob_filename = 'scale_prob.dill'
    modified_symbols_filename = "modified_symbols.txt"
    modified_symbol_types_filename = "modified_symbol_types.txt"
    modified_global_objective_filename = 'modified_global_objective_file.yaml'
    #modified_area_objectives_filename = 'modified_area_objectives_file.yaml'

    # Reload scale problem
    with open(scale_prob_filename, 'rb') as file:
        modified_scale_prob = dill.load(file)

    # Load modified constraints
    with open(modified_constraints_filename, 'r') as file:
        modified_constraints = [line.strip() for line in file]

    # Load modified constraints
    with open(modified_symbols_filename, 'r') as file:
        modified_symbols = [line.strip() for line in file]

    # Load modified constraints
    with open(modified_symbol_types_filename, 'r') as file:
        modified_symbol_types = [line.strip() for line in file]

    '''
    with open(modified_global_objective_filename, 'r') as file:
        modified_global_objective = yaml.safe_load(file)
    '''

    # Overwrite symbols
    modified_scale_prob['simplified_symbols'] = modified_symbols

    # Overwrite symbol tyeps
    modified_scale_prob['simplified_symbol_types'] = modified_symbol_types

    # Overwrite constraints list in scale problem
    modified_scale_prob['simplified_constraints'] = modified_constraints

    # Overwrite energy objectives
    # modified_scale_prob['global_objective'] = modified_global_objective

    return modified_scale_prob

In [9]:
def augment_constraint_system(scale_prob,additional_constraints):
    scale_prob['simplified_constraints']=scale_prob['simplified_constraints']+additional_constraints
    #print(scale_prob['simplified_constraints'])
    return scale_prob

In [10]:
def update_scale_prob_for_scenario(scenario_dir,scenario_constraints,add_new_symbols=False,ignore_scenario=False,modified_constraints_filename = "modified_constraints.txt"):
    modified_scale_prob = load_C_C_bidir_skip_constraint_system(modified_constraints_filename=modified_constraints_filename)
    additional_constraints = []
    transfer_relations = []
    if not ignore_scenario:
        #print("Loading scenario transfer relations")
        transfer_relations = load_lines_from_txt_file(os.path.join(scenario_dir,"transfer_relations.txt"))
        #for reln in transfer_relations:
            #print(reln)
    global_objective = load_objective_from_yaml_file(os.path.join(scenario_dir,"global_objective_file.yaml"))
    new_symbols = []
    new_symbol_types = []
    if add_new_symbols:
        new_symbols = load_lines_from_txt_file(os.path.join(scenario_dir,"symbols.txt"))
        new_symbol_types = load_lines_from_txt_file(os.path.join(scenario_dir,"symbol_types.txt"))

        #print("New symbols:")
        #print(new_symbols)
        #print("New symbol types:")
        #print(new_symbol_types)
    #print("Intersection additional constraints:")
    additional_constraints = transfer_relations+scenario_constraints
    #for cnstr in additional_constraints:
        #print(cnstr)
    modified_scale_prob = augment_constraint_system(modified_scale_prob,additional_constraints)
    modified_scale_prob['global_objective'] = global_objective
    modified_scale_prob['simplified_symbols'] = modified_scale_prob['simplified_symbols'] + new_symbols
    modified_scale_prob['simplified_symbol_types'] = modified_scale_prob['simplified_symbol_types'] + new_symbol_types
    return modified_scale_prob

In [11]:
def constraints_from_parameters(nc,sparsity_fraction0,sparsity_fraction1,ww,max_rw_ww_ratio,force_rw_max,pgen_out_thrpt):
    recip_density0 = str(1/(1-sparsity_fraction0))
    recip_density1 = str(1/(1-sparsity_fraction1))
    res = [
            "TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_nc == "+str(nc),
            "TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in1_nc == "+str(nc),
            "TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_ww == "+str(ww),
            "TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in1_ww == "+str(ww)
        ]

    if force_rw_max:
        res.extend([
            "TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_rw == "+str(ww*max_rw_ww_ratio),
            "TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in1_rw == "+str(ww*max_rw_ww_ratio)
        ])
    else:
        res.extend([
            "TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_rw <= "+str(ww*max_rw_ww_ratio),
            "TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in1_rw <= "+str(ww*max_rw_ww_ratio),
            "TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_rw >= "+str(ww),
            "TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in1_rw >= "+str(ww)
        ])

    return res

## Scenario

## Helper functions

In [41]:
def get_objective_and_transfer_relations(scenario_dir):
    #scenario_dir="C_C_bidir_direct_mapped" # C_C_bidir_direct_mapped, C_C_bidir_skip_ahead, C_C_bidir_two_finger

    #print("Loading scenario transfer relations")
    transfer_relations = load_lines_from_txt_file(os.path.join(scenario_dir,"transfer_relations.txt"))
    print(transfer_relations)
    #print("Loading global objective")
    global_objective = load_objective_from_yaml_file(os.path.join(scenario_dir,"global_objective_file.yaml"))
    #print(global_objective)
    
    # Generate sympy-safe expressions
    safe_transfer_relation_strs = [cc_.create_safe_constraint(trln_str) for trln_str in transfer_relations]
    safe_transfer_relation_strs_sparsities = \
        [safe_trln_str.replace( \
            "( TestArchitecture__Skipping_shared_buffer3_shared_buffer1__IntersectionBD__md_in0_pr )/( TestArchitecture__Skipping_shared_buffer3_shared_buffer1__IntersectionBD__md_in0_cr )", \
                "TestArchitecture__Skipping_shared_buffer3_shared_buffer1__IntersectionBD__md_in0_density_fraction0").replace( \
            "( TestArchitecture__Skipping_shared_buffer3_shared_buffer1__IntersectionBD__md_in1_pr )/( TestArchitecture__Skipping_shared_buffer3_shared_buffer1__IntersectionBD__md_in1_cr )", \
                "TestArchitecture__Skipping_shared_buffer3_shared_buffer1__IntersectionBD__md_in0_density_fraction1").replace( \
            "TestArchitecture__Skipping_shared_buffer3_shared_buffer1__IntersectionBD__md_out_pr", \
            "TestArchitecture__Skipping_shared_buffer3_shared_buffer1__IntersectionBD__md_out_pr_per_stage"
                ) \
                for safe_trln_str in safe_transfer_relation_strs]
    safe_transfer_relations_sp = [sp.sympify(safe_trln_str) for safe_trln_str in safe_transfer_relation_strs_sparsities]
    safe_global_objective_str = cc_.create_safe_constraint(global_objective)
    safe_global_objective_sp = sp.sympify(safe_global_objective_str)

    return safe_transfer_relations_sp, safe_global_objective_sp

In [70]:
def create_lambda_functions(safe_transfer_relations_sp, TestArch):
    # Define symbols for the arguments
    nc, sparsity_fraction0, sparsity_fraction1, md_in_vectorization = sp.symbols('nc sparsity_fraction0 sparsity_fraction1 md_in_vectorization')

    subs_mapping = {
        'TestArchitecture__Skipping_shared_buffer3_shared_buffer1__IntersectionBD__md_out_vectorization': 1,
        'TestArchitecture__Skipping_shared_buffer3_shared_buffer1__IntersectionBD__md_in0_nc': nc,
        'TestArchitecture__Skipping_shared_buffer3_shared_buffer1__IntersectionBD__md_in0_density_fraction0': 1.0 - sparsity_fraction0,
        'TestArchitecture__Skipping_shared_buffer3_shared_buffer1__IntersectionBD__md_in0_density_fraction1': 1.0 - sparsity_fraction1,
        'TestArchitecture__Skipping_shared_buffer3_shared_buffer1__IntersectionBD__md_in0_vectorization': md_in_vectorization
    }

    lambdas = []
    for ineq in safe_transfer_relations_sp:
        # Sympify the inequality
        ineq_expr = sp.sympify(ineq)

        # Determine which side of the inequality contains TestArch
        bound_expr = ineq_expr.rhs if sp.Symbol(TestArch) in ineq_expr.lhs.free_symbols else ineq_expr.lhs

        #print(ineq_expr)
        #print(bound_expr)
        #print(bound_expr.subs(subs_mapping))

        # Create a lambda function with 'numpy' for fast numerical computation
        lambda_fn = sp.lambdify([nc, sparsity_fraction0, sparsity_fraction1, md_in_vectorization], bound_expr.subs(subs_mapping), modules=["numpy"])
        lambdas.append(lambda_fn)

    meta_lambda = \
        lambda nc_val,sparsity_fraction0_val,sparsity_fraction1_val,md_in_vectorization_val: \
            np.min([lambda_fn(nc_val, sparsity_fraction0_val, sparsity_fraction1_val, md_in_vectorization_val) for lambda_fn in lambdas])
    return meta_lambda

In [71]:
def solve_max_throughput_per_pipeline_stage(nc, \
                                            sparsity_fraction0, \
                                            sparsity_fraction1, \
                                            md_in_vectorization, \
                                            lambda_fn):
    
    return lambda_fn(nc, sparsity_fraction0, sparsity_fraction1, md_in_vectorization)

In [78]:
def create_objective_lambda(safe_global_objective_sp_dict,intersection_unit):
    # Define symbols for the arguments
    nc, sparsity_fraction0, sparsity_fraction1, md_in_vectorization, ww, nc_thresh = sp.symbols('nc sparsity_fraction0 sparsity_fraction1 md_in_vectorization ww nc_thresh')

    # Retrieve and sympify the expression
    expr = sp.sympify(safe_global_objective_sp_dict[intersection_unit])
    
    # Extract symbols
    symbols_in_expr = expr.free_symbols

    # Define substitution mapping
    subs_mapping = {
        sp.Symbol('TestArchitecture__Skipping_shared_buffer3_shared_buffer1__IntersectionBD__md_out_vectorization'): 1,
        sp.Symbol('TestArchitecture__Skipping_shared_buffer3_shared_buffer1__IntersectionBD__md_in0_nc'): nc,
        sp.Symbol('TestArchitecture__Skipping_shared_buffer3_shared_buffer1__IntersectionBD__md_in0_density_fraction0'): 1.0 - sparsity_fraction0,
        sp.Symbol('TestArchitecture__Skipping_shared_buffer3_shared_buffer1__IntersectionBD__md_in0_density_fraction1'): 1.0 - sparsity_fraction1,
        sp.Symbol('TestArchitecture__Skipping_shared_buffer3_shared_buffer1__IntersectionBD__md_in0_vectorization'): md_in_vectorization,
        sp.Symbol('TestArchitecture__Skipping_shared_buffer3_shared_buffer1__IntersectionBD__md_in0_ww_thresh'): ww,
        sp.Symbol('TestArchitecture__Skipping_shared_buffer3_shared_buffer1__IntersectionBD__md_out_nc_thresh'): nc_thresh
    }

    # Update subs_mapping to map any remaining symbols to 0
    for sym in symbols_in_expr:
        if sym not in subs_mapping:
            if 'IntersectionBD' not in str(sym):
                subs_mapping[sym] = 0
            else:
                print("Symbol "+str(sym)+" contains IntersectionBD")
                assert(False)

    # Apply substitutions
    pruned_expr = expr.subs(subs_mapping)

    return sp.lambdify([nc, sparsity_fraction0, sparsity_fraction1, md_in_vectorization, ww, nc_thresh], pruned_expr.subs(subs_mapping), modules=["numpy"])

In [72]:
def process_expression(safe_global_objective_sp_dict,intersection_unit, nc, sparsity_fraction0, sparsity_fraction1, md_in_vectorization, ww, nc_thresh):
    # Retrieve and sympify the expression
    expr = sp.sympify(safe_global_objective_sp_dict[intersection_unit])
    
    # Extract symbols
    symbols_in_expr = expr.free_symbols

    # Define substitution mapping
    subs_mapping = {
        sp.Symbol('TestArchitecture__Skipping_shared_buffer3_shared_buffer1__IntersectionBD__md_out_vectorization'): 1,
        sp.Symbol('TestArchitecture__Skipping_shared_buffer3_shared_buffer1__IntersectionBD__md_in0_nc'): nc,
        sp.Symbol('TestArchitecture__Skipping_shared_buffer3_shared_buffer1__IntersectionBD__md_in0_density_fraction0'): 1.0 - sparsity_fraction0,
        sp.Symbol('TestArchitecture__Skipping_shared_buffer3_shared_buffer1__IntersectionBD__md_in0_density_fraction1'): 1.0 - sparsity_fraction1,
        sp.Symbol('TestArchitecture__Skipping_shared_buffer3_shared_buffer1__IntersectionBD__md_in0_vectorization'): md_in_vectorization,
        sp.Symbol('TestArchitecture__Skipping_shared_buffer3_shared_buffer1__IntersectionBD__md_in0_ww_thresh'): ww,
        sp.Symbol('TestArchitecture__Skipping_shared_buffer3_shared_buffer1__IntersectionBD__md_out_nc_thresh'): nc_thresh
    }

    # Update subs_mapping to map any remaining symbols to 0
    for sym in symbols_in_expr:
        if sym not in subs_mapping:
            if 'IntersectionBD' not in str(sym):
                subs_mapping[sym] = 0
            else:
                print("Symbol "+str(sym)+" contains IntersectionBD")
                assert(False)

    # Apply substitutions
    pruned_expr = expr.subs(subs_mapping)

    # Evaluate to a floating point value
    pruned_safe_objective_float = float(pruned_expr)

    return pruned_safe_objective_float

In [104]:
def get_num_stages(target_thrpt,ideal_matches,real_matches_per_stage,md_in_vectorization):
    num_stages = math.ceil(target_thrpt/real_matches_per_stage)
    if target_thrpt > ideal_matches or num_stages > md_in_vectorization:
        return False, None, None
    else:
        return True, num_stages, num_stages*real_matches_per_stage

## Get ideal match-count

## Main loop

In [68]:
intersection_units = ["C_C_bidir_direct_mapped","C_C_bidir_two_finger","C_C_bidir_skip_ahead"]

In [79]:
TestArch='TestArchitecture__Skipping_shared_buffer3_shared_buffer1__IntersectionBD__md_out_pr_per_stage'
safe_transfer_relations_sp_dict = {}
safe_global_objective_sp_dict = {}
lambda_dict = {}
obj_lambda_dict = {}

for isect in intersection_units:
    isect_safe_transfer_relations_sp, isect_safe_global_objective_sp = get_objective_and_transfer_relations(isect)
    safe_transfer_relations_sp_dict[isect]=isect_safe_transfer_relations_sp
    safe_global_objective_sp_dict[isect]=isect_safe_global_objective_sp
    lambda_dict[isect] = create_lambda_functions(isect_safe_transfer_relations_sp, TestArch)
    obj_lambda_dict[isect] = create_objective_lambda(safe_global_objective_sp_dict,isect)
print(lambda_dict)
print(obj_lambda_dict)

['( TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_out_pr ) <= 0.0959369126156345*( TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_out_vectorization )*(( TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_nc )*( TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_pr )/( TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_cr ))**0.488237372753155*(( TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_nc )*( TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in1_pr )/( TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in1_cr ))**0.52267185623028 + (1000*(( TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_nc ) - 8)**2 + 1000*(( TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_vectorization ) - 1)**2)/((( TestArchitecture.Skipping

In [105]:
ww=4

test_case_list=[
                    {'nc':nc,'md_in_vectorization':md_in_vectorization, \
                     'sparsity_fraction0':sparsity_fraction0, \
                     'sparsity_fraction1':sparsity_fraction1, \
                     'isect':isect, \
                     'target_thrpt':target_thrpt}
                    for target_thrpt in [0.0000001] + list(range(1)) \
                        for nc in [8, 16, 32] \
                            for md_in_vectorization in [1,2,4] \
                                for sparsity_fraction0 in [0.3,0.4,0.5,0.6,0.7] \
                                    for sparsity_fraction1 in [0.3,0.4,0.5,0.6,0.7] \
                                        for isect in intersection_units
                ]
result_list=[]
ideal_max_value=None
isect_feasible=None
num_stages=None
mean_thrpt=None
for test_case in tqdm(test_case_list):
    nc=test_case['nc']
    md_in_vectorization=test_case['md_in_vectorization']
    sparsity_fraction0=test_case['sparsity_fraction0']
    sparsity_fraction1=test_case['sparsity_fraction1']
    isect=test_case['isect']
    target_thrpt=test_case['target_thrpt']

    isect_max_value = \
        solve_max_throughput_per_pipeline_stage(nc, \
                                                sparsity_fraction0, \
                                                sparsity_fraction1, \
                                                md_in_vectorization, \
                                                lambda_dict[isect])

    if isect=="C_C_bidir_direct_mapped":
        ideal_max_value = isect_max_value
        if ideal_max_value >= target_thrpt:
            isect_feasible=True
            num_stages=1
            mean_thrpt=ideal_max_value
        else:
            isect_feasible=False
            num_stages=None
            mean_thrpt=None
    else:
        isect_feasible,num_stages,mean_thrpt = get_num_stages(target_thrpt,ideal_max_value,isect_max_value,md_in_vectorization)

    obj_val=None
    if isect_feasible:
        obj_val=obj_lambda_dict[isect](nc, sparsity_fraction0, sparsity_fraction1, md_in_vectorization, ww, nc) #process_expression(safe_global_objective_sp_dict,isect, nc, sparsity_fraction0, sparsity_fraction1, md_in_vectorization, ww, nc)

    test_case={'nc':nc, \
               'md_in_vectorization':md_in_vectorization, \
               'sparsity_fraction0':sparsity_fraction0, \
               'sparsity_fraction1':sparsity_fraction1, \
               'isect':isect, \
               'target_thrpt':target_thrpt
    }
    result={'isect_max_value':isect_max_value, \
            'isect_feasible':isect_feasible, \
            'num_stages':num_stages, \
            'mean_thrpt':mean_thrpt, \
            'obj_val':obj_val, \
            'test_case':test_case}
    result_list.append(result)

100%|██████████| 3375/3375 [00:00<00:00, 103378.90it/s]


In [106]:
def combine_test_cases_and_results(test_case_list, result_list):
    """
    Combines test cases and results into a single pandas DataFrame.

    Parameters:
    test_case_list (list of dict): List of test cases.
    result_list (list of dict): List of results corresponding to the test cases.

    Returns:
    pd.DataFrame: A DataFrame with combined test case and result data.
    """
    # Check if the lengths of the lists match
    if len(test_case_list) != len(result_list):
        raise ValueError("Length of test_case_list and result_list must be the same.")

    # Combine test cases and results into a single list of dictionaries
    combined_data = []
    for test_case, result in zip(test_case_list, result_list):
        combined_entry = {**test_case, **result}
        combined_data.append({k:combined_entry[k] for k in combined_entry if k!="test_case"})

    # Create a pandas DataFrame from the combined data
    df = pd.DataFrame(combined_data)

    # Replace None with NaN
    df.fillna(value=pd.NA, inplace=True)

    return df

In [107]:
df=combine_test_cases_and_results(test_case_list, result_list)

In [108]:
df

,nc,md_in_vectorization,sparsity_fraction0,sparsity_fraction1,isect,target_thrpt,isect_max_value,isect_feasible,num_stages,mean_thrpt,obj_val
0,8,1,0.3,0.3,C_C_bidir_direct_mapped,1.000000e-07,0.547439,True,1.0,0.547439,45.311718
1,8,1,0.3,0.3,C_C_bidir_two_finger,1.000000e-07,0.545043,True,1.0,0.545043,28.418405
2,8,1,0.3,0.3,C_C_bidir_skip_ahead,1.000000e-07,0.538722,True,1.0,0.538722,249.783692
3,8,1,0.3,0.4,C_C_bidir_direct_mapped,1.000000e-07,0.505062,True,1.0,0.505062,45.311718
4,8,1,0.3,0.4,C_C_bidir_two_finger,1.000000e-07,0.504957,True,1.0,0.504957,28.418405
...,...,...,...,...,...,...,...,...,...,...,...
3370,32,4,0.7,0.6,C_C_bidir_two_finger,2.000000e+00,0.160798,False,NaN,NaN,NaN
3371,32,4,0.7,0.6,C_C_bidir_skip_ahead,2.000000e+00,0.297227,False,NaN,NaN,NaN
3372,32,4,0.7,0.7,C_C_bidir_direct_mapped,2.000000e+00,0.599884,False,NaN,NaN,NaN
3373,32,4,0.7,0.7,C_C_bidir_two_finger,2.000000e+00,0.140805,False,NaN,NaN,NaN


In [110]:
df_filtered_thrpt = df[df['target_thrpt'] > 1]
df_filtered_thrpt

,nc,md_in_vectorization,sparsity_fraction0,sparsity_fraction1,isect,target_thrpt,isect_max_value,isect_feasible,num_stages,mean_thrpt,obj_val
2025,8,1,0.3,0.3,C_C_bidir_direct_mapped,1.5,0.547439,False,NaN,NaN,NaN
2026,8,1,0.3,0.3,C_C_bidir_two_finger,1.5,0.545043,False,NaN,NaN,NaN
2027,8,1,0.3,0.3,C_C_bidir_skip_ahead,1.5,0.538722,False,NaN,NaN,NaN
2028,8,1,0.3,0.4,C_C_bidir_direct_mapped,1.5,0.505062,False,NaN,NaN,NaN
2029,8,1,0.3,0.4,C_C_bidir_two_finger,1.5,0.504957,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3370,32,4,0.7,0.6,C_C_bidir_two_finger,2.0,0.160798,False,NaN,NaN,NaN
3371,32,4,0.7,0.6,C_C_bidir_skip_ahead,2.0,0.297227,False,NaN,NaN,NaN
3372,32,4,0.7,0.7,C_C_bidir_direct_mapped,2.0,0.599884,False,NaN,NaN,NaN
3373,32,4,0.7,0.7,C_C_bidir_two_finger,2.0,0.140805,False,NaN,NaN,NaN


In [112]:
df_filtered_feasible = df[df['isect_feasible'] == False]
df_filtered_feasible

,nc,md_in_vectorization,sparsity_fraction0,sparsity_fraction1,isect,target_thrpt,isect_max_value,isect_feasible,num_stages,mean_thrpt,obj_val
680,8,1,0.3,0.4,C_C_bidir_skip_ahead,0.5,0.497627,False,NaN,NaN,NaN
681,8,1,0.3,0.5,C_C_bidir_direct_mapped,0.5,0.459155,False,NaN,NaN,NaN
682,8,1,0.3,0.5,C_C_bidir_two_finger,0.5,0.461334,False,NaN,NaN,NaN
683,8,1,0.3,0.5,C_C_bidir_skip_ahead,0.5,0.453050,False,NaN,NaN,NaN
684,8,1,0.3,0.6,C_C_bidir_direct_mapped,0.5,0.408608,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3370,32,4,0.7,0.6,C_C_bidir_two_finger,2.0,0.160798,False,NaN,NaN,NaN
3371,32,4,0.7,0.6,C_C_bidir_skip_ahead,2.0,0.297227,False,NaN,NaN,NaN
3372,32,4,0.7,0.7,C_C_bidir_direct_mapped,2.0,0.599884,False,NaN,NaN,NaN
3373,32,4,0.7,0.7,C_C_bidir_two_finger,2.0,0.140805,False,NaN,NaN,NaN


In [86]:
result_list

[{'isect_max_value': 0.5474392344528926,
  'isect_feasible': True,
  'num_stages': 1,
  'mean_thrpt': 0.5474392344528926,
  'obj_val': 45.311717926588486,
  'test_case': {'nc': 8,
   'md_in_vectorization': 1,
   'sparsity_fraction0': 0.3,
   'sparsity_fraction1': 0.3,
   'isect': 'C_C_bidir_direct_mapped',
   'target_thrpt': 1e-07}},
 {'isect_max_value': 0.5450428791675184,
  'isect_feasible': True,
  'num_stages': 1,
  'mean_thrpt': 0.5450428791675184,
  'obj_val': 28.41840458610474,
  'test_case': {'nc': 8,
   'md_in_vectorization': 1,
   'sparsity_fraction0': 0.3,
   'sparsity_fraction1': 0.3,
   'isect': 'C_C_bidir_two_finger',
   'target_thrpt': 1e-07}},
 {'isect_max_value': 0.5387218858833922,
  'isect_feasible': True,
  'num_stages': 1,
  'mean_thrpt': 0.5387218858833922,
  'obj_val': 249.7836915475811,
  'test_case': {'nc': 8,
   'md_in_vectorization': 1,
   'sparsity_fraction0': 0.3,
   'sparsity_fraction1': 0.3,
   'isect': 'C_C_bidir_skip_ahead',
   'target_thrpt': 1e-07}},


## Brute force: direct-mapped intersection

In [22]:
## TODO

## Brute force: two-fingered intersection

In [34]:
two_finger_safe_transfer_relations_sp, two_finger_safe_global_objective_sp = get_objective_and_transfer_relations("C_C_bidir_two_finger")

# Output throughput
two_finger_max_value,two_finger_common_solution_set = \
    solve_max_throughput_per_pipeline_stage(nc, \
                                            sparsity_fraction0, \
                                            sparsity_fraction1, \
                                            md_in_vectorization, \
                                            two_finger_safe_transfer_relations_sp, \
                                            two_finger_safe_global_objective_sp)


feasible,num_stages = get_num_stages(target_thrpt,ideal_max_value,two_finger_max_value)

print(target_thrpt)
print(ideal_max_value)
print(two_finger_max_value)
print(float(two_finger_max_value)/float(ideal_max_value))
print(math.ceil(two_finger_max_value/ideal_max_value))
print(num_stages)

Loading scenario transfer relations
['( TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_out_pr ) <= 0.0950580315324327*( TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_out_vectorization )*(( TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_nc )*( TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_pr )/( TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_cr ))**0.518142991457437*(( TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_nc )*( TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in1_pr )/( TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in1_cr ))**0.495561887596442 + (1000*(( TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_nc ) - 8)**2 + 1000*(( TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_vectorization ) -

## Brute force: skip-ahead intersection

In [24]:
skip_ahead_safe_transfer_relations_sp, skip_ahead_safe_global_objective_sp = get_objective_and_transfer_relations("C_C_bidir_skip_ahead")

# Output throughput
skip_ahead_max_value,skip_ahead_common_solution_set = \
    solve_max_throughput_per_pipeline_stage(nc, \
                                            sparsity_fraction0, \
                                            sparsity_fraction1, \
                                            md_in_vectorization, \
                                            skip_ahead_safe_transfer_relations_sp, \
                                            skip_ahead_safe_global_objective_sp)

Loading scenario transfer relations
['( TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_out_pr ) <= 0.0943753613930909*( TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_out_vectorization )*(( TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_nc )*( TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_pr )/( TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_cr ))**0.496370518347299*(( TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_nc )*( TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in1_pr )/( TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in1_cr ))**0.514746960638289 + (1000*(( TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_nc ) - 8)**2 + 1000*(( TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_vectorization ) -

## Scenario: direct-mapped intersection

In [13]:
scenario_dir="C_C_bidir_direct_mapped"
scenario_constraints=constraints_from_parameters(nc,sparsity_fraction0,sparsity_fraction1,4,4,False,target_thrpt)
for s in scenario_constraints:
    print(s)
modified_scale_prob=update_scale_prob_for_scenario(scenario_dir,scenario_constraints=scenario_constraints, add_new_symbols=True)

TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_nc == 8
TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in1_nc == 8
TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_ww == 4
TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in1_ww == 4
TestArchitecture.Skipping_shared_buffer3_shared_buffer1.SinglePgenFollower.pos_out_pr >= 1e-08
TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_cr == 2.0*TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_pr
TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in1_cr == 2.0*TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in1_pr
TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_cr == TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in1_cr
TestArchitecture.Skipping_shared_buffer3_shared_buffer1.In

In [14]:
# Solve scale inference problem
abstract_analytical_primitive_models_dict,abstract_analytical_component_models_dict= \
    safcore.solve_scale_inference_problem(modified_scale_prob)

Solving scale inference problem.
model.x4 >= model.x11
model.x69 >= model.x63
model.x36 >= model.x101
model.x6*model.x90 == model.x92
model.x104 == 1
model.x4 >= model.x69*model.x36
model.x90 == model.x12
model.x92 == model.x28
model.x66 >= model.x15
model.x9 >= model.x34
model.x85 >= model.x106
model.x83*model.x21 == model.x94
model.x46 == 1
model.x66 >= model.x9*model.x85
model.x21 == model.x80
model.x94 == model.x86
model.x106 == model.x25
model.x73 >= model.x41
model.x62 >= model.x22
model.x95 >= model.x35
model.x61*model.x47 == model.x57
model.x65 == 1
model.x73 >= model.x62*model.x95
model.x47 == model.x59
model.x57 == model.x20
model.x29 >= model.x79
model.x70 >= model.x96
model.x52 >= model.x44
model.x100*model.x76 == model.x75
model.x64 == 1
model.x29 >= model.x70*model.x52
model.x76 == model.x55
model.x75 == model.x99
model.x44 == model.x40
model.x53 >= model.x16
model.x50 >= model.x87
model.x45 >= model.x7
model.x5 >= model.x27
model.x51 >= model.x23
model.x72 >= model.x60
m

ActionManagerError: Problem executing an event.  No results are available.

## Scenario: two-finger intersection

In [31]:
nc=8
target_thrpt=0.1
shared_sparsity=1/2
sparsity_fraction0=shared_sparsity
sparsity_fraction1=shared_sparsity

In [32]:
scenario_dir="C_C_bidir_two_finger"
scenario_constraints=constraints_from_parameters(nc,sparsity_fraction0,sparsity_fraction1,4,4,False,target_thrpt)
for s in scenario_constraints:
    print(s)
modified_scale_prob=update_scale_prob_for_scenario(scenario_dir,scenario_constraints=scenario_constraints, add_new_symbols=True)

TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_nc == 8
TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in1_nc == 8
TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_ww == 4
TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in1_ww == 4
TestArchitecture.Skipping_shared_buffer3_shared_buffer1.SinglePgenFollower.pos_out_pr >= 0.1
TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_cr == 2.0*TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_pr
TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in1_cr == 2.0*TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in1_pr
TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_cr == TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in1_cr
TestArchitecture.Skipping_shared_buffer3_shared_buffer1.Inte

In [33]:
# Solve scale inference problem
abstract_analytical_primitive_models_dict,abstract_analytical_component_models_dict= \
    safcore.solve_scale_inference_problem(modified_scale_prob)

Solving scale inference problem.
model.x4 >= model.x11
model.x69 >= model.x63
model.x36 >= model.x101
model.x6*model.x90 == model.x92
model.x104 == 1
model.x4 >= model.x69*model.x36
model.x90 == model.x12
model.x92 == model.x28
model.x66 >= model.x15
model.x9 >= model.x34
model.x85 >= model.x106
model.x83*model.x21 == model.x94
model.x46 == 1
model.x66 >= model.x9*model.x85
model.x21 == model.x80
model.x94 == model.x86
model.x106 == model.x25
model.x73 >= model.x41
model.x62 >= model.x22
model.x95 >= model.x35
model.x61*model.x47 == model.x57
model.x65 == 1
model.x73 >= model.x62*model.x95
model.x47 == model.x59
model.x57 == model.x20
model.x29 >= model.x79
model.x70 >= model.x96
model.x52 >= model.x44
model.x100*model.x76 == model.x75
model.x64 == 1
model.x29 >= model.x70*model.x52
model.x76 == model.x55
model.x75 == model.x99
model.x44 == model.x40
model.x53 >= model.x16
model.x50 >= model.x87
model.x45 >= model.x7
model.x5 >= model.x27
model.x51 >= model.x23
model.x72 >= model.x60
m

ActionManagerError: Problem executing an event.  No results are available.

## Scenario: skip-ahead intersection

In [46]:
nc=8
target_thrpt=0.0000001
shared_sparsity=0.5
sparsity_fraction0=shared_sparsity
sparsity_fraction1=shared_sparsity

In [47]:
scenario_dir="C_C_bidir_skip_ahead"
scenario_constraints=constraints_from_parameters(nc,sparsity_fraction0,sparsity_fraction1,4,4,False,target_thrpt)
for s in scenario_constraints:
    print(s)
modified_scale_prob=update_scale_prob_for_scenario(scenario_dir,scenario_constraints=scenario_constraints, add_new_symbols=True)

TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_nc == 8
TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in1_nc == 8
TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_ww == 4
TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in1_ww == 4
TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_rw <= 16
TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in1_rw <= 16
TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in0_rw >= 4
TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_in1_rw >= 4
Loading scenario transfer relations
( TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_out_pr ) <= 0.0943753613930909*( TestArchitecture.Skipping_shared_buffer3_shared_buffer1.IntersectionBD.md_out_vectorization )*(( TestArchitecture.Skipping_shared_buffer3_shared_buffer1.Intersection

In [48]:
# Solve scale inference problem
abstract_analytical_primitive_models_dict,abstract_analytical_component_models_dict= \
    safcore.solve_scale_inference_problem(modified_scale_prob)

Solving scale inference problem.
model.x4 >= model.x11
model.x69 >= model.x63
model.x36 >= model.x101
model.x6*model.x90 == model.x92
model.x104 == 1
model.x4 >= model.x69*model.x36
model.x90 == model.x12
model.x92 == model.x28
model.x66 >= model.x15
model.x9 >= model.x34
model.x85 >= model.x106
model.x83*model.x21 == model.x94
model.x46 == 1
model.x66 >= model.x9*model.x85
model.x21 == model.x80
model.x94 == model.x86
model.x106 == model.x25
model.x73 >= model.x41
model.x62 >= model.x22
model.x95 >= model.x35
model.x61*model.x47 == model.x57
model.x65 == 1
model.x73 >= model.x62*model.x95
model.x47 == model.x59
model.x57 == model.x20
model.x29 >= model.x79
model.x70 >= model.x96
model.x52 >= model.x44
model.x100*model.x76 == model.x75
model.x64 == 1
model.x29 >= model.x70*model.x52
model.x76 == model.x55
model.x75 == model.x99
model.x44 == model.x40
model.x53 >= model.x16
model.x50 >= model.x87
model.x45 >= model.x7
model.x5 >= model.x27
model.x51 >= model.x23
model.x72 >= model.x60
m

ActionManagerError: Problem executing an event.  No results are available.

## Special naive intersection case

In [ ]:
scenario_dir="C_C_bidir_two_finger"
#scenario_constraints=constraints_from_parameters(nc,sparsity_fraction0,sparsity_fraction1,4,4,False,target_thrpt)
#for s in scenario_constraints:
#    print(s)
modified_scale_prob=update_scale_prob_for_scenario(scenario_dir,scenario_constraints=[], add_new_symbols=True, ignore_scenario=True,modified_constraints_filename="special_modified_constraints.txt")

NameError: name 'update_scale_prob_for_scenario' is not defined

In [ ]:
# Solve scale inference problem
abstract_analytical_primitive_models_dict,abstract_analytical_component_models_dict= \
    safcore.solve_scale_inference_problem(modified_scale_prob)

NameError: name 'modified_scale_prob' is not defined

## End

In [ ]:
assert(False)

In [18]:
!rm ref_output/arch_w_SAF ref_output/arch_w_SAF_C_bidir.yaml ref_output/bindings_C_bidir.yaml
!safmodel -T ref_output/new_arch_C_bidir.yaml -a ref_input/arch.yaml -s ref_input/sparseopts_C_bidir.yaml -c ref_input/compound_components.yaml -U ref_input/safmodel_settings.yaml  -r ref_output/arch_w_SAF.yaml -k   ref_output/ -L
!mv ref_output/arch_w_SAF.yaml ref_output/arch_w_SAF_C_bidir.yaml
!mv safmodel.log safmodel_C_bidir.log

rm: ref_output/arch_w_SAF: No such file or directory
rm: ref_output/arch_w_SAF_C_bidir.yaml: No such file or directory
rm: ref_output/bindings_C_bidir.yaml: No such file or directory


SAFmodel.

Parsing input files:
- netlist: ref_output/new_arch_C_bidir.yaml
- arch: ref_input/arch.yaml
- sparseopts: ref_input/sparseopts_C_bidir.yaml
- compound components (input): ['ref_input/compound_components.yaml']
- arch output path: ref_output/arch_w_SAF.yaml
- compound components path (output): ref_output/
- SAFModel settings path: ref_input/safmodel_settings.yaml
logging: True
>> SAFmodel
:: Setup
:: Scale inference
Building scale inference problem...
Building RegisteredEqualityComparator_bitwidth$(u) energy area latency table.: 1
- RegisteredEqualityComparator_bitwidth$(u) MSE energy error (symbolic vs numerical): 1.525611648803889e-31
- RegisteredEqualityComparator_bitwidth$(u) RNMSE energy error (numerical vs true): 8.4 %
- RegisteredEqualityComparator_bitwidth$(u) MSE area error (symbolic vs numerical): 9.707226444223633e-25
- RegisteredEqualityComparator_bitwidth$(u) RNMSE area error (numerical vs true): 2.6 %
Building RegisteredEqualityComparator_bitwidth$(u) energy ar

In [16]:
!rm ref_output/arch_w_SAF ref_output/arch_w_SAF_C_lf.yaml ref_output/bindings_C_lf.yaml
!safmodel -T ref_output/new_arch_C_lf.yaml -a ref_input/arch.yaml -s ref_input/sparseopts_C_lf.yaml -c ref_input/compound_components.yaml -U ref_input/safmodel_settings.yaml  -r ref_output/arch_w_SAF.yaml -k   ref_output/ -L
!mv ref_output/arch_w_SAF.yaml ref_output/arch_w_SAF_C_lf.yaml
!mv safmodel.log safmodel_C_lf.log

rm: ref_output/arch_w_SAF: No such file or directory
rm: ref_output/bindings_C_lf.yaml: No such file or directory


SAFmodel.

Parsing input files:
- netlist: ref_output/new_arch_C_lf.yaml
- arch: ref_input/arch.yaml
- sparseopts: ref_input/sparseopts_C_lf.yaml
- compound components (input): ['ref_input/compound_components.yaml']
- arch output path: ref_output/arch_w_SAF.yaml
- compound components path (output): ref_output/
- SAFModel settings path: ref_input/safmodel_settings.yaml
logging: True
>> SAFmodel
:: Setup
:: Scale inference
Building scale inference problem...
Building RegisteredEqualityComparator_bitwidth$(u) energy area latency table.: 1
- RegisteredEqualityComparator_bitwidth$(u) MSE energy error (symbolic vs numerical): 1.525611648803889e-31
- RegisteredEqualityComparator_bitwidth$(u) RNMSE energy error (numerical vs true): 8.4 %
- RegisteredEqualityComparator_bitwidth$(u) MSE area error (symbolic vs numerical): 9.707226444223633e-25
- RegisteredEqualityComparator_bitwidth$(u) RNMSE area error (numerical vs true): 2.6 %
Building RegisteredEqualityComparator_bitwidth$(u) energy area lat